In [11]:
from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI, OpenAI

# read local .env file
# find_dotenv 函数通常用于搜索和定位目录树中的 .env 文件，而 load_dotenv 函数用于将 .env 文件中的变量加载到环境中。
_ = load_dotenv(find_dotenv())

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3)
# embeddings = OpenAIEmbeddings()

In [12]:
"""
该教程涵盖了如何将链序列化到磁盘并从磁盘反序列化。我们使用的序列化格式是json或yaml。目前仅有一些链支持此类型的序列化。我们将逐步增加支持的链数。
"""
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
template = """Question: {question}
 
Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)
 

In [13]:
llm_chain.save("files/llm_chain.json")
 

In [14]:
# !cat files/llm_chain.json

"""
{
    "name": null,
    "memory": null,
    "verbose": true,
    "tags": null,
    "metadata": null,
    "prompt": {
        "name": null,
        "input_variables": [
            "question"
        ],
        "input_types": {},
        "output_parser": null,
        "partial_variables": {},
        "template": "Question: {question}\n \nAnswer: Let's think step by step.",
        "template_format": "f-string",
        "validate_template": false,
        "_type": "prompt"
    },
    "llm": {
        "model_name": "gpt-3.5-turbo",
        "model": "gpt-3.5-turbo",
        "stream": false,
        "n": 1,
        "temperature": 0.3,
        "_type": "openai-chat"
    },
    "output_key": "text",
    "output_parser": {
        "name": null,
        "_type": "default"
    },
    "return_final_only": true,
    "llm_kwargs": {},
    "_type": "llm_chain"
}
"""

'\n{\n    "name": null,\n    "memory": null,\n    "verbose": true,\n    "tags": null,\n    "metadata": null,\n    "prompt": {\n        "name": null,\n        "input_variables": [\n            "question"\n        ],\n        "input_types": {},\n        "output_parser": null,\n        "partial_variables": {},\n        "template": "Question: {question}\n \nAnswer: Let\'s think step by step.",\n        "template_format": "f-string",\n        "validate_template": false,\n        "_type": "prompt"\n    },\n    "llm": {\n        "model_name": "gpt-3.5-turbo",\n        "model": "gpt-3.5-turbo",\n        "stream": false,\n        "n": 1,\n        "temperature": 0.3,\n        "_type": "openai-chat"\n    },\n    "output_key": "text",\n    "output_parser": {\n        "name": null,\n        "_type": "default"\n    },\n    "return_final_only": true,\n    "llm_kwargs": {},\n    "_type": "llm_chain"\n}\n'

In [15]:
"""
从磁盘加载链
20240128 ValueError: Loading openai-chat LLM not supported
"""
from langchain.chains import load_chain
"""

"""
chain = load_chain("files/llm_chain.json")


ValueError: Loading openai-chat LLM not supported

In [17]:
chain.invoke("whats 2 + 2")




> Entering new LLMChain chain...
Prompt after formatting:
Question: whats 2 + 2
 
Answer: Let's think step by step.


APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [ ]:
"""
分别保存组件
在上面的例子中，我们可以看到提示和llm配置信息保存在与整个链相同的json中。
或者，我们可以将它们拆分并分别保存。这通常有助于使保存的组件更加模块化。为了做到这一点，我们只需要指定llm_path而不是llm组件，以及prompt_path而不是prompt组件。
"""
llm_chain.prompt.save("files/prompt.json")
llm_chain.llm.save("files/llm.json")


In [1]:
config = {
    "memory": None,
    "verbose": True,
    "prompt_path": "prompt.json",
    "llm_path": "llm.json",
    "output_key": "text",
    "_type": "llm_chain"
}
import json
with open("files/llm_chain_separate.json", "w") as f:
    json.dump(config, f, indent=2)
 

In [ ]:
"""
然后我们可以以相同的方式加载它。
"""
chain = load_chain("llm_chain_separate.json")
chain.invoke("whats 2 + 2")
